In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [6]:
from langchain.tools import tool
from typing import Dict, Any
from tavily import TavilyClient
from langchain_community.utilities import SQLDatabase

tavily_client = TavilyClient()

db = SQLDatabase.from_uri("sqlite:///../advanced-agents/resources/Chinook.db")


@tool
def web_search(query: str) -> Dict[str, Any]:

    """Search the web for information"""

    return tavily_client.search(query)

@tool
def sql_query(query: str) -> str:

    """Obtain information from the database using SQL queries"""

    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

In [7]:
from dataclasses import dataclass

@dataclass
class UserRole:
    user_role: str = "external"

In [8]:
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from typing import Callable

@wrap_model_call
def dynamic_tool_call(request: ModelRequest, 
handler: Callable[[ModelRequest], ModelResponse]) -> ModelResponse:

    """Dynamically call tools based on the runtime context"""

    user_role = request.runtime.context.user_role
    
    if user_role == "internal":
        pass # internal users get access to all tools
    else:
        tools = [web_search] # external users only get access to web search
        request = request.override(tools=tools) 

    return handler(request)

In [9]:
from langchain.agents import create_agent

agent = create_agent(
    model="gpt-5-nano",
    tools=[web_search, sql_query],
    middleware=[dynamic_tool_call],
    context_schema=UserRole
)

In [10]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"messages": [HumanMessage(content="How many artists are in the database?")]},
    context={"user_role": "external"}
)

print(response["messages"][-1].content)

I don’t have access to your database, so I can’t tell you the exact count. Please tell me which database and table you’re referring to, or share the schema. If you want, I can provide the exact query.

Common options:
- SQL (e.g., PostgreSQL, MySQL)
  - If there is a dedicated artists table: SELECT COUNT(*) AS artist_count FROM artists;
  - If you want to count unique artists by id: SELECT COUNT(DISTINCT id) AS artist_count FROM artists;
  - If artists are identified by name: SELECT COUNT(DISTINCT name) AS artist_count FROM artists;
- MongoDB (NoSQL)
  - db.artists.countDocuments({})
  - Or if counting distinct IDs: db.artists.distinct("_id").length
- CSV/flat file
  - If you have a CSV with an artist_id column: use a script to count distinct IDs, e.g., in bash: cut -d, -f<id_field> artists.csv | sort -u | wc -l

If you share the exact database type and table/field names (and whether you want unique artists only), I’ll write a precise query for you.
